# Bitcoin Price Prediction - Data Exploration

This notebook explores the Bitcoin price data, performs initial analysis, and visualizes key patterns.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add the project root to Python path
sys.path.append(os.path.join(os.getcwd(), '..'))

from config.config import DATA_CONFIG
from utils.data_loader import BitcoinDataLoader

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

## 1. Load Bitcoin Data

In [ ]:
# Initialize data loader
data_loader = BitcoinDataLoader(DATA_CONFIG)

# Load data
df = data_loader.load_data()

print("📊 Data Overview:")
print(f"Shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"Total days: {len(df)}")
print("\nColumns:", list(df.columns))
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Basic statistics
print("📈 Basic Statistics:")
df.describe()

## 2. Price Analysis

In [ ]:
# Plot price history
plt.figure(figsize=(14, 8))

plt.subplot(2, 2, 1)
plt.plot(df.index, df['close'], linewidth=2, color='#FF9500')
plt.title('Bitcoin Closing Price Over Time', fontweight='bold', fontsize=14)
plt.ylabel('Price (USD)')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
plt.bar(df.index, df['volume'], alpha=0.7, color='#007AFF')
plt.title('Trading Volume Over Time', fontweight='bold', fontsize=14)
plt.ylabel('Volume')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
daily_returns = df['close'].pct_change() * 100
plt.hist(daily_returns.dropna(), bins=50, alpha=0.7, color='#34C759')
plt.title('Daily Returns Distribution', fontweight='bold', fontsize=14)
plt.xlabel('Daily Return (%)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 4)
price_range = (df['high'] - df['low']) / df['close'] * 100
plt.plot(df.index, price_range, alpha=0.7, color='#AF52DE')
plt.title('Daily Price Range (% of Close)', fontweight='bold', fontsize=14)
plt.ylabel('Price Range (%)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Volatility Analysis

In [ ]:
# Calculate rolling volatility
df['daily_return'] = df['close'].pct_change()
df['volatility_7d'] = df['daily_return'].rolling(window=7).std() * np.sqrt(365) * 100  # Annualized
df['volatility_30d'] = df['daily_return'].rolling(window=30).std() * np.sqrt(365) * 100

plt.figure(figsize=(14, 10))

plt.subplot(3, 1, 1)
plt.plot(df.index, df['close'], linewidth=2, color='#FF9500')
plt.title('Bitcoin Price with Volatility Regimes', fontweight='bold', fontsize=16)
plt.ylabel('Price (USD)')
plt.grid(True, alpha=0.3)

plt.subplot(3, 1, 2)
plt.plot(df.index, df['volatility_7d'], label='7-day Volatility', linewidth=2, color='#FF3B30')
plt.plot(df.index, df['volatility_30d'], label='30-day Volatility', linewidth=2, color='#5856D6')
plt.title('Rolling Volatility (Annualized %)', fontweight='bold', fontsize=14)
plt.ylabel('Volatility (%)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(3, 1, 3)
# Identify high volatility periods (top 20%)
high_vol_threshold = df['volatility_30d'].quantile(0.8)
high_vol_periods = df['volatility_30d'] > high_vol_threshold

plt.scatter(df.index[~high_vol_periods], df['daily_return'][~high_vol_periods] * 100, 
           alpha=0.5, color='gray', label='Normal Volatility')
plt.scatter(df.index[high_vol_periods], df['daily_return'][high_vol_periods] * 100, 
           alpha=0.7, color='red', label='High Volatility')
plt.title('Daily Returns Colored by Volatility Regime', fontweight='bold', fontsize=14)
plt.ylabel('Daily Return (%)')
plt.xlabel('Date')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📊 Volatility Statistics:")
print(f"Average 7-day volatility: {df['volatility_7d'].mean():.2f}%")
print(f"Average 30-day volatility: {df['volatility_30d'].mean():.2f}%")
print(f"High volatility threshold (80th percentile): {high_vol_threshold:.2f}%")

## 4. Correlation Analysis

In [ ]:
# Calculate correlations
correlation_matrix = df[['open', 'high', 'low', 'close', 'volume', 'daily_return']].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f', linewidths=0.5)
plt.title('Price and Volume Correlations', fontweight='bold', fontsize=16)
plt.tight_layout()
plt.show()

## 5. Seasonality and Trends

In [ ]:
# Monthly performance analysis
df['month'] = df.index.month
df['year'] = df.index.year
monthly_returns = df.groupby('month')['daily_return'].mean() * 100

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
colors = ['red' if x < 0 else 'green' for x in monthly_returns]
bars = plt.bar(monthly_returns.index, monthly_returns, color=colors, alpha=0.7)
plt.title('Average Monthly Returns (%)', fontweight='bold', fontsize=14)
plt.xlabel('Month')
plt.ylabel('Average Return (%)')
plt.grid(True, alpha=0.3)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}%', ha='center', va='bottom' if height >= 0 else 'top')

plt.subplot(1, 2, 2)
# Year-over-year performance
yearly_perf = df.groupby('year')['close'].apply(lambda x: (x.iloc[-1] / x.iloc[0] - 1) * 100)
colors = ['red' if x < 0 else 'green' for x in yearly_perf]
bars = plt.bar(yearly_perf.index.astype(str), yearly_perf, color=colors, alpha=0.7)
plt.title('Yearly Performance (%)', fontweight='bold', fontsize=14)
plt.xlabel('Year')
plt.ylabel('Total Return (%)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.1f}%', ha='center', va='bottom' if height >= 0 else 'top')

plt.tight_layout()
plt.show()

## 6. Data Quality Check

In [ ]:
print("🔍 Data Quality Report:")
print("=" * 50)

# Check for missing values
print("\nMissing Values:")
missing_data = df.isnull().sum()
for col, missing in missing_data.items():
    if missing > 0:
        print(f"  {col}: {missing} missing values ({missing/len(df)*100:.2f}%)")

# Check for duplicates
print(f"\nDuplicate rows: {df.duplicated().sum()}")

# Check data types
print("\nData Types:")
print(df.dtypes)

# Check for outliers in returns
returns = df['daily_return'].dropna()
q1 = returns.quantile(0.01)
q99 = returns.quantile(0.99)
outliers = returns[(returns < q1) | (returns > q99)]
print(f"\nExtreme returns (outside 1st-99th percentile): {len(outliers)} ({len(outliers)/len(returns)*100:.2f}%)")

# Check for zero or negative prices
invalid_prices = df[(df['close'] <= 0) | (df['open'] <= 0) | (df['high'] <= 0) | (df['low'] <= 0)]
print(f"Invalid prices (<= 0): {len(invalid_prices)}")

print("\n✅ Data exploration complete!")

## Key Insights from Data Exploration

1. **Price Trends**: Bitcoin shows significant volatility with clear bull/bear market cycles
2. **Volatility Patterns**: Periods of high volatility cluster together
3. **Correlations**: OHLC prices are highly correlated, as expected
4. **Seasonality**: Some monthly patterns may exist in returns
5. **Data Quality**: Good overall with minimal missing values

Next: Proceed to feature engineering in the next notebook.